In [10]:
import pandas as pd

In [11]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

#display(raw_data)

In [12]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [13]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [14]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

MP   W  D   L  MW  ML   GF   GA  GD  Par%  W%  Pts       Form
Gameweek 33                                                               
Calz         19  11  1   7   5   1  289  241  48    57  57  6.5  ➖ 🔼 🔽 🔼 🔽
Reaper       27  17  0  10   3   5  380  376   4    81  62  6.0  🔽 🔼 ⏫ 🔼 🔼
Schurrle     20  12  0   8   4   2  305  275  30    60  60  5.0  🔼 🔼 🔽 ⏫ 🔽
Gazprom      22  13  0   9   2   3  308  305   3    66  59  3.5  🔼 🔽 🔽 🔼 🔼
Virlo        13   7  1   5   2   2  191  186   5    39  53  2.5  ➖ 🔽 🔽 🔼 🔼
Keysey       14   7  1   6   2   1  202  184  18    42  50  2.0  🔽 🔽 ⏫ 🔼 🔼
Zola         18   9  1   8   3   3  249  265 -16    54  50  1.5  🔽 🔽 🔼 🔼 ⏬
Peggy         6   3  0   3   1   0   90   79  11    18  50  0.5  🔽 🔼 ⏫ 🔽 🔼
Gilet        17   7  0  10   3   3  237  253 -16    51  41 -3.0  🔼 🔽 🔼 ⏫ 🔽
Golfer        9   3  0   6   1   2  118  127  -9    27  33 -3.5  🔽 🔽 🔼 ⏫ 🔽
Helmet       25  11  0  14   3   5  332  367 -35    75  44 -4.0  🔼 🔽 ⏬ 🔽 🔽
Kim          29  12  1  16   3   4  394  426 -32    87  41 -4.0  🔽 🔽 ⏬ 🔼 🔼
Snoop        21   8  0  13   3   3  280  284  -4    63  38 -5.0  🔼 🔼 ⏬ 🔽 🔽
Noel         18   5  1  12   2   5  223  275 -52    54  27 -8.0  ⏬ ➖ 🔼 🔽 🔽
Mate         28   9  1  18   4   4  385  394  -9    84  32 -8.5  🔽 🔽 🔽 ⏬ 🔽

In [15]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [16]:
display(raw_data)

Name       Date  GF  GA     Team  GD  GamePoints  W  D  L  MW  ML  \
0      Calz 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
1    Helmet 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
2       Kim 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
3      Mate 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
4     Peggy 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
..      ...        ...  ..  ..      ...  ..         ... .. .. ..  ..  ..   
325    Calz 2023-10-05  11  14  BLANCOS  -3        -1.0  0  0  1   0   0   
326   Gilet 2023-10-05  11  14  BLANCOS  -3        -1.0  0  0  1   0   0   
327    Mate 2023-10-05  11  14  BLANCOS  -3        -1.0  0  0  1   0   0   
328   Snoop 2023-10-05  11  14  BLANCOS  -3        -1.0  0  0  1   0   0   
329  Helmet 2023-10-05  11  14  BLANCOS  -3        -1.0  0  0  1   0   0   

    Result  game_rank  
0        🔽       19.0  
1        🔽       25.0  
2        🔽       29.0  
3        🔽       28.0  
4        🔽        6.0  
..     ...        ...  
325      🔽        1.0  
326      🔽        1.0  
327      🔽        1.0  
328      🔽        1.0  
329      🔽        1.0  

[286 rows x 14 columns]

In [17]:
def count_same_team(df, name_col, date_col, team_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            print(same_team_names)
    return count

In [18]:
count_same_team(raw_data, 'Name', 'Date', 'Team')

['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 

['Kim' 'Mate' 'Reaper' 'Golfer']
['Kim' 'Mate' 'Reaper' 'Golfer']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Gilet' 'Keysey' 'Golfer' 'Reaper' 'Schurrle']
['Gilet' 'Keysey' 'Golfer' 'Reaper' 'Schurrle']
['Gilet' 'Keysey' 'Golfer' 'Reaper' 'Schurrle']
['Gilet' 'Keysey' 'Golfer' 'Reaper' 'Schurrle']
['Gilet' 'Keysey' 'Golfer' 'Reaper' 'Schurrle']
['Zola' 'Mate' 'Snoop' 'Kim' 'Helmet']
['Zola' 'Mate' 'Snoop' 'Kim' 'Helmet']
['Zola' 'Mate' 'Snoop' 'Kim' 'Helmet']
['Zola' 'Mate' 'Snoop' 'Kim' 'Helmet']
['Zola' 'Mate' 'Snoop' 'Kim' 'Helmet']
['Keysey' 'Kim' 'Reaper' 'Gazprom']
['Keysey' 'Kim' 'Reaper' 'Gazprom']
['Keysey' 'Kim' 'Reaper' 'Gazprom']
['Keysey' 'Kim' 'Reaper' 'Gazprom']
['Golfer' 'Snoop' 'Helmet' 'Schurrle']
['Golfer' 'Snoop' 'Helmet' 'Schurrle

0